In [1]:
#Generates a set of coefficients as a sage "vector"
def coefs_1d(N,N0,lab) :
    return vector([ var(lab+'%s'%i) for i in range(N0,N0+N) ])


#Generates a 1-D polynomial as a sage expreession based on a set of coefficients
def poly_1d(N,coefs,x) :
    return sum( vector([ coefs[i]*x^i for i in range(N) ]) )


#Force the 'expr' to be in floating point with the given precision
def force_fp(expr,prec) :
    R = RealField( prec )
    expr = expr * R(2.) / R(2.)
    return expr


#Single scalar value with a single array of coefficients
def c_scalar(retlab,val,coeflab) :
    import re
    code = ""
    s = retlab+" = "
    #Remove spaces
    s = s+str(val).replace(' ','')
    #Replace squares with multiplication
    s = re.sub("([a-zA-Z()0-9_]*)\\^2","(\\1*\\1)",s,0,re.DOTALL)
    #Replace exponentials with power functions
    s = re.sub("([a-zA-Z()0-9_]*)\\^([0-9]*)","pow((double)\\1,(double)\\2)",s,0,re.DOTALL)
    #Replace coeficients (e.g., c1) with parentheses (e.g., c(1))
    s = re.sub(coeflab+"([0-9]+)",coeflab+"(\\1)",s, 0, re.DOTALL)
    #Add _fp suffixes to floating point numbers
    s = re.sub("([0-9]*\\.[0-9e-]*)","\\1_fp",s, 0, re.DOTALL)
    #Add new line to the end.
    code = code + s + ";\n"
    return code


In [2]:
# Left
coefs = coefs_1d(2,0,'a')
p = poly_1d(2,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pL = poly_1d(2,constr_to_coefs*coefs_1d(2,0,'s'),x)

# Right
coefs = coefs_1d(2,0,'a')
p = poly_1d(2,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pR = poly_1d(2,constr_to_coefs*coefs_1d(2,1,'s'),x)

# High
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(3,constr_to_coefs*coefs_1d(3,0,'s'),x)
print(c_scalar('qL',force_fp(pH.subs(x=-1/2),65),'s'))
print(c_scalar('qR',force_fp(pH.subs(x= 1/2),65),'s'))

var('gamma_L,gamma_R')
# Ideal weights at -1/2
xloc = -1/2
stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+pR.subs(x=xloc)*gamma_R,coefs_1d(3,0,'s'))
A = jacobian(stencil_weights,[gamma_L,gamma_R])
rhs = Matrix(3,1,jacobian(pH.subs(x=xloc),coefs_1d(3,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")
# Ideal weights at 1/2
xloc = 1/2
stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+pR.subs(x=xloc)*gamma_R,coefs_1d(3,0,'s'))
A = jacobian(stencil_weights,[gamma_L,gamma_R])
rhs = Matrix(3,1,jacobian(pH.subs(x=xloc),coefs_1d(3,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")

var('w_L,w_R')
print(c_scalar('qL',force_fp((pL*w_L+pR*w_R).subs(x=-1/2).simplify_full(),65),'s'))
print(c_scalar('qR',force_fp((pL*w_L+pR*w_R).subs(x= 1/2).simplify_full(),65),'s'))


qL = 0.3333333333333333333_fp*s(0)+0.8333333333333333333_fp*s(1)-0.1666666666666666667_fp*s(2);

qR = -0.1666666666666666667_fp*s(0)+0.8333333333333333333_fp*s(1)+0.3333333333333333333_fp*s(2);

[0.6666666666666666667]
[0.3333333333333333333] 

[0.3333333333333333333]
[0.6666666666666666667] 

qL = 0.5000000000000000000_fp*(s(0)+s(1))*w_L+0.5000000000000000000_fp*(3*s(1)-s(2))*w_R;

qR = -0.5000000000000000000_fp*(s(0)-3*s(1))*w_L+0.5000000000000000000_fp*(s(1)+s(2))*w_R;



In [3]:
# Left
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pL = poly_1d(3,constr_to_coefs*coefs_1d(3,0,'s'),x)

# Center
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pC = poly_1d(3,constr_to_coefs*coefs_1d(3,1,'s'),x)

# Right
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pR = poly_1d(3,constr_to_coefs*coefs_1d(3,2,'s'),x)

# High
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(5,constr_to_coefs*coefs_1d(5,0,'s'),x)
print(c_scalar('qL',force_fp(pH.subs(x=-1/2),65),'s'))
print(c_scalar('qR',force_fp(pH.subs(x= 1/2),65),'s'))

var('gamma_L,gamma_C,gamma_R')
# Ideal weights at -1/2
xloc = -1/2
stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+
                           pC.subs(x=xloc)*gamma_C+
                           pR.subs(x=xloc)*gamma_R,coefs_1d(5,0,'s'))
A = jacobian(stencil_weights,[gamma_L,gamma_C,gamma_R])
rhs = Matrix(5,1,jacobian(pH.subs(x=xloc),coefs_1d(5,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")
# Ideal weights at 1/2
xloc = 1/2
stencil_weights = jacobian(pL.subs(x=xloc)*gamma_L+
                           pC.subs(x=xloc)*gamma_C+
                           pR.subs(x=xloc)*gamma_R,coefs_1d(5,0,'s'))
A = jacobian(stencil_weights,[gamma_L,gamma_C,gamma_R])
rhs = Matrix(5,1,jacobian(pH.subs(x=xloc),coefs_1d(5,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")

var('w_L,w_C,w_R')
print(c_scalar('qL',force_fp((pL*w_L+pC*w_C+pR*w_R).subs(x=-1/2).simplify_full(),65),'s'))
print(c_scalar('qR',force_fp((pL*w_L+pC*w_C+pR*w_R).subs(x= 1/2).simplify_full(),65),'s'))

qL = -0.05000000000000000000_fp*s(0)+0.4500000000000000000_fp*s(1)+0.7833333333333333333_fp*s(2)-0.2166666666666666667_fp*s(3)+0.03333333333333333333_fp*s(4);

qR = 0.03333333333333333333_fp*s(0)-0.2166666666666666667_fp*s(1)+0.7833333333333333333_fp*s(2)+0.4500000000000000000_fp*s(3)-0.05000000000000000000_fp*s(4);

[0.3000000000000000000]
[0.6000000000000000000]
[0.1000000000000000000] 

[0.1000000000000000000]
[0.6000000000000000000]
[0.3000000000000000000] 

qL = 0.1666666666666666667_fp*(2*s(1)+5*s(2)-s(3))*w_C-0.1666666666666666667_fp*(s(0)-5*s(1)-2*s(2))*w_L+0.1666666666666666667_fp*(11*s(2)-7*s(3)+2*s(4))*w_R;

qR = -0.1666666666666666667_fp*(s(1)-5*s(2)-2*s(3))*w_C+0.1666666666666666667_fp*(2*s(0)-7*s(1)+11*s(2))*w_L+0.1666666666666666667_fp*(2*s(2)+5*s(3)-s(4))*w_R;



In [4]:
# 1
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p1 = poly_1d(4,constr_to_coefs*coefs_1d(4,0,'s'),x)

# 2
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p2 = poly_1d(4,constr_to_coefs*coefs_1d(4,1,'s'),x)

# 3
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p3 = poly_1d(4,constr_to_coefs*coefs_1d(4,2,'s'),x)

# 4
coefs = coefs_1d(4,0,'a')
p = poly_1d(4,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p4 = poly_1d(4,constr_to_coefs*coefs_1d(4,3,'s'),x)

# High
coefs = coefs_1d(7,0,'a')
p = poly_1d(7,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(7,constr_to_coefs*coefs_1d(7,0,'s'),x)
print(c_scalar('qL',force_fp(pH.subs(x=-1/2),65),'s'))
print(c_scalar('qR',force_fp(pH.subs(x= 1/2),65),'s'))

var('gamma_1,gamma_2,gamma_3,gamma_4')
# Ideal weights at -1/2
xloc = -1/2
stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                           p2.subs(x=xloc)*gamma_2+
                           p3.subs(x=xloc)*gamma_3+
                           p4.subs(x=xloc)*gamma_4,coefs_1d(7,0,'s'))
A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4])
rhs = Matrix(7,1,jacobian(pH.subs(x=xloc),coefs_1d(7,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")
# Ideal weights at 1/2
xloc = 1/2
stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                           p2.subs(x=xloc)*gamma_2+
                           p3.subs(x=xloc)*gamma_3+
                           p4.subs(x=xloc)*gamma_4,coefs_1d(7,0,'s'))
A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4])
rhs = Matrix(7,1,jacobian(pH.subs(x=xloc),coefs_1d(7,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")

var('w_1,w_2,w_3,w_4')
print(c_scalar('qL',force_fp((p1*w_1+p2*w_2+p3*w_3+p4*w_4).subs(x=-1/2).simplify_full(),65),'s'))
print(c_scalar('qR',force_fp((p1*w_1+p2*w_2+p3*w_3+p4*w_4).subs(x= 1/2).simplify_full(),65),'s'))

qL = 0.009523809523809523810_fp*s(0)-0.09047619047619047619_fp*s(1)+0.5095238095238095238_fp*s(2)+0.7595238095238095238_fp*s(3)-0.2404761904761904762_fp*s(4)+0.05952380952380952381_fp*s(5)-0.007142857142857142857_fp*s(6);

qR = -0.007142857142857142857_fp*s(0)+0.05952380952380952381_fp*s(1)-0.2404761904761904762_fp*s(2)+0.7595238095238095238_fp*s(3)+0.5095238095238095238_fp*s(4)-0.09047619047619047619_fp*s(5)+0.009523809523809523810_fp*s(6);

[ 0.1142857142857142857]
[ 0.5142857142857142857]
[ 0.3428571428571428571]
[0.02857142857142857143] 

[0.02857142857142857143]
[ 0.3428571428571428571]
[ 0.5142857142857142857]
[ 0.1142857142857142857] 

qL = 0.08333333333333333333_fp*(s(0)-5*s(1)+13*s(2)+3*s(3))*w_1-0.08333333333333333333_fp*(s(1)-7*s(2)-7*s(3)+s(4))*w_2+0.08333333333333333333_fp*(3*s(2)+13*s(3)-5*s(4)+s(5))*w_3+0.08333333333333333333_fp*(25*s(3)-23*s(4)+13*s(5)-3*s(6))*w_4;

qR = -0.08333333333333333333_fp*(3*s(0)-13*s(1)+23*s(2)-25*s(3))*w_1+0.08333333333333333333_fp*(s(1)-5*s(

In [5]:
# 1
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-9/2,-7/2) ,
                  integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p1 = poly_1d(5,constr_to_coefs*coefs_1d(5,0,'s'),x)

# 2
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p2 = poly_1d(5,constr_to_coefs*coefs_1d(5,1,'s'),x)

# 3
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p3 = poly_1d(5,constr_to_coefs*coefs_1d(5,2,'s'),x)

# 4
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p4 = poly_1d(5,constr_to_coefs*coefs_1d(5,3,'s'),x)

# 5
coefs = coefs_1d(5,0,'a')
p = poly_1d(5,coefs,x)
constr = vector([ integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ,
                  integrate(p,x, 7/2, 9/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
p5 = poly_1d(5,constr_to_coefs*coefs_1d(5,4,'s'),x)

# High
coefs = coefs_1d(9,0,'a')
p = poly_1d(9,coefs,x)
constr = vector([ integrate(p,x,-9/2,-7/2) ,
                  integrate(p,x,-7/2,-5/2) ,
                  integrate(p,x,-5/2,-3/2) ,
                  integrate(p,x,-3/2,-1/2) ,
                  integrate(p,x,-1/2, 1/2) ,
                  integrate(p,x, 1/2, 3/2) ,
                  integrate(p,x, 3/2, 5/2) ,
                  integrate(p,x, 5/2, 7/2) ,
                  integrate(p,x, 7/2, 9/2) ])
constr_to_coefs = jacobian(constr,coefs)^-1
pH = poly_1d(9,constr_to_coefs*coefs_1d(9,0,'s'),x)
print(c_scalar('qL',force_fp(pH.subs(x=-1/2),65),'s'))
print(c_scalar('qR',force_fp(pH.subs(x= 1/2),65),'s'))

var('gamma_1,gamma_2,gamma_3,gamma_4,gamma_5')
# Ideal weights at -1/2
xloc = -1/2
stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                           p2.subs(x=xloc)*gamma_2+
                           p3.subs(x=xloc)*gamma_3+
                           p4.subs(x=xloc)*gamma_4+
                           p5.subs(x=xloc)*gamma_5,coefs_1d(9,0,'s'))
A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4,gamma_5])
rhs = Matrix(9,1,jacobian(pH.subs(x=xloc),coefs_1d(9,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")
# Ideal weights at 1/2
xloc = 1/2
stencil_weights = jacobian(p1.subs(x=xloc)*gamma_1+
                           p2.subs(x=xloc)*gamma_2+
                           p3.subs(x=xloc)*gamma_3+
                           p4.subs(x=xloc)*gamma_4+
                           p5.subs(x=xloc)*gamma_5,coefs_1d(9,0,'s'))
A = jacobian(stencil_weights,[gamma_1,gamma_2,gamma_3,gamma_4,gamma_5])
rhs = Matrix(9,1,jacobian(pH.subs(x=xloc),coefs_1d(9,0,'s')).list())
print(force_fp((A.transpose()*A)^-1*A.transpose()*rhs,65),"\n")

var('w_1,w_2,w_3,w_4,w_5')
print(c_scalar('qL',force_fp((p1*w_1+p2*w_2+p3*w_3+p4*w_4+p5*w_5).subs(x=-1/2).simplify_full(),65),'s'))
print(c_scalar('qR',force_fp((p1*w_1+p2*w_2+p3*w_3+p4*w_4+p5*w_5).subs(x= 1/2).simplify_full(),65),'s'))

qL = -0.001984126984126984127_fp*s(0)+0.02182539682539682540_fp*s(1)-0.1210317460317460317_fp*s(2)+0.5456349206349206349_fp*s(3)+0.7456349206349206349_fp*s(4)-0.2543650793650793651_fp*s(5)+0.07896825396825396825_fp*s(6)-0.01626984126984126984_fp*s(7)+0.001587301587301587302_fp*s(8);

qR = 0.001587301587301587302_fp*s(0)-0.01626984126984126984_fp*s(1)+0.07896825396825396825_fp*s(2)-0.2543650793650793651_fp*s(3)+0.7456349206349206349_fp*s(4)+0.5456349206349206349_fp*s(5)-0.1210317460317460317_fp*s(6)+0.02182539682539682540_fp*s(7)-0.001984126984126984127_fp*s(8);

[ 0.03968253968253968254]
[  0.3174603174603174603]
[  0.4761904761904761905]
[  0.1587301587301587302]
[0.007936507936507936508] 

[0.007936507936507936508]
[  0.1587301587301587302]
[  0.4761904761904761905]
[  0.3174603174603174603]
[ 0.03968253968253968254] 

qL = -0.01666666666666666667_fp*(3*s(0)-17*s(1)+43*s(2)-77*s(3)-12*s(4))*w_1+0.01666666666666666667_fp*(2*s(1)-13*s(2)+47*s(3)+27*s(4)-3*s(5))*w_2-0.016666666666666666